In [1]:
%%capture

# Turn off warnings
import warnings

warnings.filterwarnings("ignore")

# Normal packages
import geopandas as gpd
import numpy as np
import pandas as pd

# Format
from babel.numbers import format_currency

# Display
from IPython.display import HTML, Image, Markdown, display, display_html

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

# GCS, del later since this will presumbly be read from a script that cleans up the data
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

In [2]:
# Charts
import altair as alt
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp

In [31]:
# Parameter Cell - the county of interest
parameter_county = "Alameda"

In [32]:
# Create dfs
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [33]:
# Filter out a df for the county set in the parameter cell
df_parameter = df_statewide.loc[
    df_statewide["full_county_name"] == parameter_county
].reset_index(drop=True)

In [34]:
# Grab the district this county is in
parameter_county_district = df_parameter["district"][0]

# Grab the full district name
parameter_county_district_full_name = df_parameter["district_full_name"][0]

In [35]:
# Dataframe just for the district the parameter county is located in
df_parameter_district = df_statewide.loc[
    df_statewide["district"] == parameter_county_district
].reset_index(drop=True)

In [36]:
df_parameter_district["district_full_name"][0]

'04 - Oakland'

In [37]:
concat1 = _utils.county_district_comparison(df_parameter, df_parameter_district)

In [38]:
concat1

,Total Projects,"Total Project Cost $1,000",Current Fake Fund Requested,Geography,Total Project ($1000) Formatted,Fake Fund Formatted
0,88.00,"33,502,945.00","26,015,757.65",04 - Oakland,"$33,502,945.00","$26,015,757.65"
1,30.00,"8,126,126.00","9,042,850.83",Alameda,"$8,126,126.00","$9,042,850.83"


In [41]:
def repeated_charts(
    df,
    color_col: str,
    y_encoding_list: list,
    x_encoding_list: list,
    chart_title: str,
    tooltip_col: list,
):
    base = (
        alt.Chart()
        .mark_bar()
        .encode(
            color=alt.Color(
                color_col, scale=alt.Scale(range=cp.CALITP_DIVERGING_COLORS), 
                legend = None
            ),
            tooltip= y_encoding_list + tooltip_col,
        )
        .properties(width=150, height=100)
        .interactive()
    )

    chart = alt.vconcat(data=df)
    for y_encoding in y_encoding_list:
        row = alt.hconcat()
        for x_encoding in x_encoding_list:
            row |= base.encode(x=x_encoding, y=y_encoding)
        chart &= row

    return chart.properties(title=chart_title)

In [42]:
repeated_charts(
    concat1,
    "Geography:N",
    ["Geography:N"],
    [
        "Total Projects:Q",
        "Total Project Cost  $1,000:Q",
        "Current Fake Fund Requested:Q",
    ],
    "My Title",
    [ 
        "Total Project ($1000) Formatted",
        "Fake Fund Formatted",
        "Total Projects",
    ],
)

alt.VConcatChart(...)